In [2]:

import os
import nltk
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
from GoogleNews import GoogleNews
googlenews = GoogleNews()
import alpaca_trade_api as tradeapi
from newsapi.newsapi_client import NewsApiClient
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
import hvplot.pandas
%matplotlib inline
# Load .env enviroment variables
load_dotenv()


True

In [6]:
# Reading the News API key enviroment variable
api_key = os.getenv("Api_keys.env")

In [7]:


# Set News API Key
#newsapi = NewsApiClient(api_key=os.environ["news_api"])

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

#api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

In [8]:
load_dotenv(dotenv_path="Api_keys.env")
api_key = os.getenv("news_api")
api_key

'22b1baf9e3d84f549a60cd9ca7881995'

In [3]:
googlenews = GoogleNews(lang='en')
googlenews = GoogleNews(start='01/01/2019',end='12/30/2020')
googlenews = GoogleNews(encode='utf-8')

In [11]:
googlenews.get_news("Pfizer")
googlenews.search("Pfizer")

In [10]:
googlenews.results()

[{'title': "Apple Highlights Services in 2020 as App Store Sets Single-Day Spending Record on New Year's Day",
  'desc': "Apple has today announced that New Year's Day saw a new single-day spending record of over $540 million on the App Store, as the company...",
  'date': '10 hours ago',
  'datetime': datetime.datetime(2021, 1, 6, 14, 14, 28),
  'link': 'news.google.com/./articles/CAIiENF-vBLO4uh3BYhi5xvCy8kqFwgEKg8IACoHCAowyIXgAjCQuhsw0fkk?hl=en-US&gl=US&ceid=US%3Aen',
  'img': 'https://lh3.googleusercontent.com/tEDkONtDxIU-VY_PPR71dUPTG_BCuKXi_12ZojbkxpVjv9vkIkXliT3akA_iDSzv48IYkqv3BYH06M9UJ4Q=-p-df-h100-w100',
  'media': None,
  'site': 'MacRumors'},
 {'title': 'Apple CEO Tim Cook Earned $14.8 Million in 2020, Not Counting Stock Awards',
  'desc': 'Apple CEO Tim Cook earned more than $14 million in salary in 2020 according to the 2020 proxy statement that Apple filed with the SEC this...',
  'date': 'Yesterday',
  'datetime': datetime.datetime(2021, 1, 5, 22, 19, 36),
  'link': 'ne